In [1]:
import pandas as pd

In [ ]:
# 1-  Cargar archivo Excel
xlsx_path = "customer-details.xlsx"
excel_file = pd.ExcelFile(xlsx_path)

# 🔍 Ver nombres de hojas
print("📄 Hojas disponibles:", excel_file.sheet_names)


# Mapeo hoja -> año
mapa_hojas = {
    '2012': '2012',
    '2013': '2013',
    '2014': '2014'
}

# 🔀 Unificar todas las hojas en un solo DataFrame con columna 'año'
df_xlsx_all = pd.concat(
    [excel_file.parse(sheet).assign(año=year) for sheet, year in mapa_hojas.items()],
    ignore_index=True
)



# ✅ Renombrar columna 'ID' → 'uuid' para preparar el merge
df_xlsx_all = df_xlsx_all.rename(columns={'ID': 'uuid'})
# 🧪 Comprobar resultado
print("📎 Dataset combinado desde el Excel:")
print(f"🧮 Registros: {df_xlsx_all.shape[0]}")
print(f"📋 Columnas: {df_xlsx_all.columns.tolist()}")
print("🧪 ¿Está 'uuid' en df_xlsx_all?", 'uuid' in df_xlsx_all.columns)
print("📋 Primeras filas:", df_xlsx_all[['uuid']].head())

📄 Hojas disponibles: ['2012', '2013', '2014']
🔁 Registros completamente duplicados (todas las columnas): 0
📎 Dataset combinado desde el Excel:
🧮 Registros: 43170
📋 Columnas: ['Unnamed: 0', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'NumWebVisitsMonth', 'uuid', 'año']
🧪 ¿Está 'uuid' en df_xlsx_all? True
📋 Primeras filas:                                    uuid
0  089b39d8-e4d0-461b-87d4-814d71e0e079
1  e9d37224-cb6f-4942-98d7-46672963d097
2  3f9f49b5-e410-4948-bf6e-f9244f04918b
3  9991fafb-4447-451a-8be2-b0df6098d13e
4  eca60b76-70b6-4077-80ba-bc52e8ebb0eb


In [ ]:
# 2 — Cargar los datos CSV
with open("bank-additional.csv", "r", encoding="utf-8") as f:
    cabecera_raw = f.readline().strip()
if cabecera_raw.startswith(","):
    cabecera_raw = cabecera_raw[1:]
    columnas = [col.strip() for col in cabecera_raw.split(",")]
try:
    df_bank  = pd.read_csv(
        "bank-additional.csv",
        sep=",",
        skiprows=1,
        names=columnas,
        encoding="utf-8",
        quotechar='"',
        decimal=","
    )
    print("✅ CSV cargado con éxito")
except Exception as e:
    print("❌ Error al cargar el CSV:", e)
print(f"🔁 Registros completamente duplicados (todas las columnas): {duplicados_completos}")

# Renombrar columna id_ → uuid si aún no está hecho
df_bank = df_bank.rename(columns={'id_': 'uuid'})

print("📎 Dataset CSV:")
print(f"🧮 Registros: {df_bank.shape[0]}")
print(f"📋 Columnas: {df_bank.columns.tolist()}")


✅ CSV cargado con éxito
🔁 Registros completamente duplicados (todas las columnas): 0
📎 Dataset CSV:
🧮 Registros: 43000
📋 Columnas: ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y', 'date', 'latitude', 'longitude', 'uuid']


In [4]:
# 3 - Mergeamos

# 🔗 Realizamos el merge usando 'uuid' como clave
df_merged = pd.merge(df_bank, df_xlsx_all, on='uuid', how='outer')

# 🧪 Verificamos el resultado del merge
print("✅ Merge completado con éxito")
print(f"🔢 Registros en df_bank: {df_bank.shape[0]}")
print(f"🔢 Registros en df_xlsx_all: {df_xlsx_all.shape[0]}")
print(f"🔗 Registros después del merge: {df_merged.shape[0]}")
print(f"📋 Columnas del nuevo dataset: {df_merged.columns.tolist()}")

# 👁️ Mostramos algunas filas para verificar visualmente
display(df_merged.sample(5))

✅ Merge completado con éxito
🔢 Registros en df_bank: 43000
🔢 Registros en df_xlsx_all: 43170
🔗 Registros después del merge: 43170
📋 Columnas del nuevo dataset: ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y', 'date', 'latitude', 'longitude', 'uuid', 'Unnamed: 0', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'NumWebVisitsMonth', 'año']


,age,job,marital,education,default,housing,loan,contact,duration,campaign,...,latitude,longitude,uuid,Unnamed: 0,Income,Kidhome,Teenhome,Dt_Customer,NumWebVisitsMonth,año
27484,NaN,admin.,MARRIED,NaN,0.0,1.0,0.0,cellular,82.0,1.0,...,44.049,-92.8,a2cfa7e2-3d19-40a6-86af-3abfb779b912,8824,157590,2,0,2014-12-23,7,2014
39309,33.0,admin.,MARRIED,university.degree,0.0,0.0,0.0,cellular,168.0,1.0,...,33.86,-70.618,e9288a7f-64e2-4b8c-b101-fa2d19f3197c,19609,58839,2,2,2012-12-23,20,2012
12802,24.0,technician,SINGLE,professional.course,0.0,1.0,0.0,cellular,355.0,2.0,...,39.058,-108.26,4b9d53d9-e861-46d4-bd07-36a4ba1d61d2,11143,43559,0,2,2014-12-26,28,2014
43004,NaN,admin.,MARRIED,university.degree,0.0,0.0,0.0,cellular,135.0,1.0,...,36.946,-116.209,fef39322-8a0d-428c-8171-430adae63e88,13751,74588,1,2,2014-04-24,29,2014
6407,NaN,admin.,MARRIED,university.degree,0.0,1.0,0.0,cellular,8.0,2.0,...,40.131,-80.049,25b56bf2-7b45-44d6-b7c2-168507457b7b,6743,69981,2,0,2014-05-20,4,2014


In [5]:
# 4 - Exportar DataFrame fusionado a archivo Excel
output_path = "data_set_complete_raw.xlsx"

# 💾 Guardar sin índice, para mantenerlo limpio
df_merged.to_excel(output_path, index=False)

# ✅ Confirmación
print(f"✅ Archivo guardado con éxito en: {output_path}")

✅ Archivo guardado con éxito en: data_set_complete_raw.xlsx
